# Exploratory Data Analysis Taxi NYC

In case you do not dispose from a local environment please launch this repository from: 

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/Talan-TechForData/datascience-questions/HEAD?labpath=1_exploratory_data_analysis/taxinyc_analysis/problem.ipynb)
[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aladinoster/datascience-exercises/blob/main/notebook-practice.ipynb)

## Situational Description

Within the folder `1_exploratory_data_analysis/taxinyc_analysis/` you will find a single file containing events from  a time series, this events correspond to messages received from trains 
where each line corresponds to an event registred by the train. 

In [1]:
%pip install jupysql duckdb-engine geopandas pyarrow shapely

Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext sql
%sql duckdb://

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb://'

You may use `%%sql` to query data from the parquet file as shown in the example down below

In [3]:
%%sql
FROM read_parquet('data/train.parquet.gz')
LIMIT 2

Running query in 'duckdb://'

TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
1372636858620000589,C,None,None,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[-8.620326,41.14251],[-8.622153,41.143815],[-8.623953,41.144373],[-8.62668,41.144778],[-8.627373,41.144697],[-8.630226,41.14521],[-8.632746,41.14692],[-8.631738,41.148225],[-8.629938,41.150385],[-8.62911,41.151213],[-8.629128,41.15124],[-8.628786,41.152203],[-8.628687,41.152374],[-8.628759,41.152518],[-8.630838,41.15268],[-8.632323,41.153022],[-8.631144,41.154489],[-8.630829,41.154507],[-8.630829,41.154516],[-8.630829,41.154498],[-8.630838,41.154489]]"
1372637303620000596,B,None,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[-8.642196,41.160114],[-8.644455,41.160492],[-8.646921,41.160951],[-8.649999,41.161491],[-8.653167,41.162031],[-8.656434,41.16258],[-8.660178,41.163192],[-8.663112,41.163687],[-8.666235,41.1642],[-8.669169,41.164704],[-8.670852,41.165136],[-8.670942,41.166576],[-8.66961,41.167962],[-8.668098,41.168988],[-8.66664,41.170005],[-8.665767,41.170635],[-8.66574,41.170671]]"


1. As you may see there are some columns containin information about taxi trips. What are the types and the quantities of call types?

In [4]:
%%sql
SELECT
    CALL_TYPE,
    COUNT(*) cnt
FROM read_parquet('data/train.parquet.gz')
GROUP BY CALL_TYPE

Running query in 'duckdb://'

CALL_TYPE,cnt
A,364770
C,528019
B,817881


2. What is the ORIGIN CALL and the maximum number of trips within a day of type A (not null ones)

In [5]:
%%sql
SELECT
    ORIGIN_CALL,
    DAY_TYPE,
    COUNT(*) cnt
FROM read_parquet('data/train.parquet.gz')
GROUP BY ORIGIN_CALL, DAY_TYPE
HAVING ORIGIN_CALL IS NOT NULL
ORDER BY cnt DESC
LIMIT 5

Running query in 'duckdb://'

ORIGIN_CALL,DAY_TYPE,cnt
2002.0,A,57571
63882.0,A,6406
2001.0,A,2499
13168.0,A,1314
6728.0,A,1115


3. What is the Taxi with most cumulated trips?

In [6]:
%%sql
SELECT
    COUNT(*) trip_count,
    TAXI_ID
FROM read_parquet('data/train.parquet.gz')
GROUP BY TAXI_ID
ORDER BY trip_count DESC
LIMIT 10

Running query in 'duckdb://'

trip_count,TAXI_ID
10746,20000080
9238,20000403
8449,20000066
7821,20000364
7729,20000483
7609,20000129
7498,20000307
7276,20000621
7267,20000089
7176,20000424


4. What is the average time difference between two trips ?

In [26]:
%%sql
WITH taxi_times AS (
    SELECT
        TAXI_ID,
        to_timestamp(TIMESTAMP) as current,
        LAG(to_timestamp(TIMESTAMP)) OVER (PARTITION BY TAXI_ID ORDER BY TIMESTAMP) as previous
    FROM read_parquet('data/train.parquet.gz')
)
SELECT
    AVG(EXTRACT(SECOND FROM current - previous)) as avg_time
FROM taxi_times


Running query in 'duckdb://'

avg_time
29.40182794982172


# Fin